# ASSIGNMENT 4

## Nisan Ece Kalem, 2008472

In [19]:
import pandas as pd 
import statistics as st
import re

Since I am more comfortable with manipulating data in data frame format, I used pandas package and implement the questions accordingly

In [20]:
data = pd.read_csv("data.txt", sep = "\t")

### a. 

#### Longest protein in the dataset

In [21]:
protLengthNames = data[["Gene names", "Length"]] ## Subset of 'data' to only include  gene names and length
protLengthNames.loc[protLengthNames['Length'] == max(protLengthNames['Length'])]  ## location of protein with maximum length

,Gene names,Length
1732,TTN,34350


#### Average length of all proteins

In [22]:
st.mean(protLengthNames["Length"]) ## average length of the proteins

562.516

### b.

#### How many proteins contain the sequence “PRSTQ”? Print their entry names and the position of the given sequence.

In [23]:
protSeq = data[["Entry","Gene names", "Sequence"]]


In [24]:
def searchSequence(seq, dat):
    pattern = re.compile(seq)
    results = [re.search(pattern, x) for x in dat["Sequence"].get_values()]
    dfSeq = pd.DataFrame({"pattern": results})
    matches = dat[dfSeq["pattern"].notna()] ## eliminating the non-hit results
    return(matches)
prstq_matches = searchSequence("PRSTQ", protSeq)
print(len(prstq_matches))

0


There are no proteins that include the pattern "PRSTQ" in their sequence in this dataset. As a proof of concept, i tried my function with a pattern that does exist in the dataset, "PSTQ":

In [25]:
pstq_matches = searchSequence("PSTQ", protSeq)
pstq_matches

,Entry,Gene names,Sequence
260,O00555,CACNA1A CACH4 CACN3 CACNL1A4,MARFGDEMPARYGGGGSGAAAGVVVGSGGGRGAGGSRQGGQPGAQR...
304,Q9P243,ZFAT KIAA1485 ZFAT1 ZNF406,METRAAENTAIFMCKCCNLFSPNQSELLSHVSEKHMEEGVNVDEII...
686,A0A1B0GUV7,TEX48,MAAHQNLILKIFCLCCRDCQEPYAINDSKVPSQTQEHKPSTQNLLL...
852,Q9H3R1,NDST4 HSST4,MNLIVKLRRSFRTLIVLLATFCLVSIVISAYFLYSGYKQEMTLIET...
1070,Q9H3S1,SEMA4A SEMAB SEMB UNQ783/PRO1317,MALPALGLDPWSLLGLFLFQLLQLLLPTTTAGGGGQGPMPRVRYYA...
1333,O60264,SMARCA5 SNF2H WCRF135,MSSAAEPPPPPPPESAPSKPAASIASGGSNSSNKGGPEGVAAQAVA...
1351,Q12815,TROAP,MTTRQATKDPLLRGVSPTPSKIPVRSQKRTPFPTVTSCAVDQENQD...
1421,Q53GL7,PARP10,MVAMAEAEAGVAVEVRGLPPAVPDELLTLYFENRRRSGGGPVLSWQ...
1589,Q9H5Y7,SLITRK6,MKLWIHLFYSSLLACISLHSQTPVLSSRGSCDSLCNCEEKDGTMLI...
1657,O95487,SEC24B,MSAPAGSSHPAASARIPPKFGGAAVSGAAAPAGPGAGPAPHQQNGP...


In [26]:
print("There are %d entries in this datasets that include the pattern 'PSTQ' in their sequence" % len(pstq_matches))

There are 12 entries in this datasets that include the pattern 'PSTQ' in their sequence


### c.

#### What is the percentage of proteins having at least one 3D structure?

In [27]:
present3D = data["3D"].notna() ##Boolean array for selecting non-missing values for "3D" column
with3Dstr = data[present3D]

In [28]:
percentageOf3D = (len(with3Dstr)/len(data))*100
print('Percentage of the proteins having at least one 3D structure is %.2f' % percentageOf3D)

Percentage of the proteins having at least one 3D structure is 31.55


#### Ratio of (total x-ray structures) / (total NMR structures)

In [29]:
xrayRegEx = r"(X-ray crystallography)\s+\((\d+)\)" ## regex pattern with grouping to capture only the number later on
noXrays = [re.findall(xrayRegEx, x) for x in with3Dstr["3D"]]
noXray = filter(None, noXrays) ## Eliminating empty, non-hit results
qtXray = [int(x[0][1]) for x in noXray] # Capturing the group with number
sumofXray =sum(qtXray)

In [30]:
nmrRegEx = r"(NMR spectroscopy)\s+\((\d+)\)" ## regex pattern with grouping to capture only the number later on
noNMRs = [re.findall(nmrRegEx, x) for x in with3Dstr["3D"]]
noNMR = filter(None, noNMRs) ## Eliminating empty, non-hit results
qtNMR = [int(x[0][1]) for x in noNMR] # Capturing the group with number 
sumofNMR =sum(qtNMR)

In [31]:
ratio = sumofXray / sumofNMR
print('The ratio of X-ray crystallographies to NMR spectroscopy is %.2f' % ratio)

The ratio of X-ray crystallographies to NMR spectroscopy is 10.57


#### Unique set of methods for 3D structure

In [32]:
pattern = r"(.+)\s+\((\d+)\)"
uniqueMethods = []
for struc in with3Dstr["3D"]:
    tmp = [re.findall(pattern, x) for x in struc.split(";")] ## splitting different methods that may exist for a single entry
    uniqueMethods.append(tmp)
uniqueMethods = filter(None, uniqueMethods) ## Eliminating empty results
uniqueMethods = set([x[0][0][0] for x in uniqueMethods])
print(uniqueMethods)

{'NMR spectroscopy', 'X-ray crystallography', 'Electron microscopy', 'Model'}


### d
#### Average (mass/length) ratio for all proteins

In [33]:
massData = [int(re.sub(",", "", x)) for x in data["Mass"]]  ## fix formating for 'Mass' column, removing ',' characters used for thousand separators
massData = pd.DataFrame({"mass": massData})

ratioData = massData['mass'] / data["Length"]

In [34]:
st.mean(ratioData)

111.59928958964974